**Cleanning the customer dat**

1. create a dataframe with proper datatyes

2. Rename a few coloumn

3. Insert a new coloumn as ingestion date(current time stamp)

4. Remove complete duplicate records

5. Remove rows with annual income as null.

6. convert emp lenght to integer

7. need to replace nulls in emp_lenght with avg og the coloumn

8. clean the address_state (should be 2 chars)

9. write cleaned customers data in cleaned folder 

In [1]:
#import the necessary libs

from pyspark.sql import SparkSession
import getpass

username=getpass.getuser()
spark=SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.suffle.useOldFetchProtocol",'true'). \
    config("spark.dql.warehouse.dir",f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
customers_df=spark.read \
    .format("csv") \
    .option("inferSchema", "True") \
    .option("header", "True") \
    .load("/user/itv014478/lendingclubproject/raw/customer_data_csv")

In [3]:
customers_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: double (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: string (nullable = true)
 |-- verification_status_joint: string (nullable = true)



**create a dataframe with proper datatyes**

In [4]:
customer_schema="member_id string,emp_title string,emp_length string,home_ownership string,annual_inc float,addr_state string,zip_code string,country string,grade string,sub_grade string,verification_status string,tot_hi_cred_lim float,application_type string,annual_inc_joint string,verification_status_joint string"

In [5]:
customers_df=spark.read \
    .format("csv") \
    .schema(customer_schema) \
    .option("header", "True") \
    .load("/user/itv014478/lendingclubproject/raw/customer_data_csv")

In [6]:
customers_df.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- annual_inc_joint: string (nullable = true)
 |-- verification_status_joint: string (nullable = true)



In [7]:
#Rename a few coloumn

customers_df_renamed=customers_df.withColumnRenamed("annual_inc", "annual_income") \
.withColumnRenamed("addr_state","address_state") \
.withColumnRenamed("zip_code","address_zipcode") \
.withColumnRenamed("country","address_country") \
.withColumnRenamed("tot_hi_cref_lim","total_high_credit_limit") \
.withColumnRenamed("annual_inc_joint","joint_annual_income")

In [8]:
customers_df_renamed.show()

+--------------------+--------------------+----------+--------------+-------------+----------+---------------+---------------+-----+---------+-------------------+---------------+----------------+-------------------+-------------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|addr_state|address_zipcode|address_country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|joint_annual_income|verification_status_joint|
+--------------------+--------------------+----------+--------------+-------------+----------+---------------+---------------+-----+---------+-------------------+---------------+----------------+-------------------+-------------------------+
|6d5091b3fcaaeb4ea...|             leadman| 10+ years|      MORTGAGE|      55000.0|        PA|          190xx|            USA|    C|       C4|       Not Verified|       178050.0|      Individual|               null|                     null|
|b5e7938b0a2da4cea...|          

In [9]:
#Insert a new coloumn as ingestion date(current time stamp)


from pyspark.sql.functions import current_timestamp
customers_df_ingested=customers_df_renamed.withColumn("ingest_date",current_timestamp())

In [10]:
customers_df_ingested.count()

462494

In [11]:
#Remove complete duplicate records
customer_df_distinct=customers_df_ingested.distinct()

In [12]:
customer_df_distinct.count()

462488

In [13]:
customer_df_distinct.createOrReplaceTempView("customers")

In [14]:
spark.sql("SELECT * FROM customers LIMIT 10")

member_id,emp_title,emp_length,home_ownership,annual_income,addr_state,address_zipcode,address_country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,joint_annual_income,verification_status_joint,ingest_date
3d6b18547c79937bf...,Program Manager,7 years,OWN,102000.0,VA,225xx,USA,C,C5,Not Verified,104252.0,Individual,null,null,2024-10-22 11:04:...
f651f7c77ad2e2d99...,Senior Analyst,10+ years,MORTGAGE,105000.0,GA,300xx,USA,F,F3,Source Verified,389349.0,Individual,null,null,2024-10-22 11:04:...
4dbf2f3d012c19d35...,Produce Manager,< 1 year,RENT,50000.0,IL,607xx,USA,B,B5,Not Verified,37550.0,Individual,null,null,2024-10-22 11:04:...
8ccefd35d022e9d58...,Ranch manager,10+ years,OWN,52000.0,CA,937xx,USA,A,A5,Verified,56500.0,Individual,null,null,2024-10-22 11:04:...
04a239bd3b937e90f...,Laborer,10+ years,MORTGAGE,100000.0,IL,611xx,USA,A,A3,Not Verified,157950.0,Individual,null,null,2024-10-22 11:04:...
ea34c8f9b3407f941...,mechanic,10+ years,MORTGAGE,80000.0,OK,740xx,USA,B,B1,Source Verified,200343.0,Individual,null,null,2024-10-22 11:04:...
b3c2ddc2956466fd6...,Store manager,7 years,RENT,37000.0,FL,323xx,USA,B,B5,Verified,124636.0,Individual,null,null,2024-10-22 11:04:...
ef56645d4be3d1fd3...,ACCOUNTS PAYABLE,10+ years,RENT,77856.0,NJ,073xx,USA,B,B4,Verified,82077.0,Individual,null,null,2024-10-22 11:04:...
18806938d2feebf62...,Associate,< 1 year,RENT,76000.0,NY,113xx,USA,A,A5,Source Verified,102194.0,Individual,null,null,2024-10-22 11:04:...
05708ce68b471b65a...,Investment Manager,3 years,RENT,160000.0,KY,402xx,USA,E,E3,Verified,113398.0,Individual,null,null,2024-10-22 11:04:...


In [15]:
#Remove rows with annual income as null.

spark.sql("SELECT COUNT(*) FROM customers WHERE annual_income is NULL")

count(1)
2


In [16]:
customer_df_filtered_income=spark.sql("SELECT * FROM customers WHERE annual_income is NOT NULL")

In [17]:
customer_df_filtered_income.createOrReplaceTempView("customers")

In [18]:
#convert emp lenght to integer

spark.sql("Select distinct(emp_length) from customers")

emp_length
9 years
5 years
null
1 year
2 years
7 years
8 years
4 years
6 years
3 years


In [19]:
from pyspark.sql.functions import regexp_replace, col


In [20]:
customer_df_emp_length_clean=customer_df_filtered_income.withColumn("emp_length", regexp_replace(col("emp_length"),"(\D)",""))

In [21]:
customer_df_emp_length_clean.show()

+--------------------+--------------------+----------+--------------+-------------+----------+---------------+---------------+-----+---------+-------------------+---------------+----------------+-------------------+-------------------------+--------------------+
|           member_id|           emp_title|emp_length|home_ownership|annual_income|addr_state|address_zipcode|address_country|grade|sub_grade|verification_status|tot_hi_cred_lim|application_type|joint_annual_income|verification_status_joint|         ingest_date|
+--------------------+--------------------+----------+--------------+-------------+----------+---------------+---------------+-----+---------+-------------------+---------------+----------------+-------------------+-------------------------+--------------------+
|8def1d73e5b3c19a7...|Speech Language P...|         2|      MORTGAGE|     75181.21|        VA|          242xx|            USA|    C|       C2|       Not Verified|       354971.0|      Individual|               n

In [22]:
#convert emp lenght to integer

customer_df_emp_length_cast=customer_df_emp_length_clean.withColumn("emp_length",customer_df_emp_length_clean.emp_length.cast('int'))

In [23]:
customer_df_emp_length_cast.printSchema()

root
 |-- member_id: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: integer (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_income: float (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- address_zipcode: string (nullable = true)
 |-- address_country: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- tot_hi_cred_lim: float (nullable = true)
 |-- application_type: string (nullable = true)
 |-- joint_annual_income: string (nullable = true)
 |-- verification_status_joint: string (nullable = true)
 |-- ingest_date: timestamp (nullable = false)



In [24]:
customer_df_emp_length_cast.filter("emp_length is null").count()

27532

In [25]:
customer_df_emp_length_cast.createOrReplaceTempView("customers")

In [32]:
avg_emp_Length=spark.sql("SELECT FLOOR(AVG(emp_length)) AS AVG_EMP_LENGTH FROM customers").collect()

In [33]:
print(avg_emp_Length)

[Row(AVG_EMP_LENGTH=6)]


In [34]:
avg_emp_duration=avg_emp_Length[0][0]

In [35]:
customer_df_emp_length_replaced=customer_df_emp_length_cast.na.fill(avg_emp_duration,subset=['emp_length'])

In [36]:
customer_df_emp_length_replaced.filter("emp_length is null").count()

0

In [39]:
# Clean the address statewith 2 chars.
customer_df_emp_length_replaced.createOrReplaceTempView("customers")


In [44]:
spark.sql("SELECT DISTINCT(addr_state) FROM customers").count()

50

In [46]:
spark.sql("SELECT count(addr_state) FROM customers WHERE length(addr_state)>2")

count(addr_state)
0


In [47]:
customer_df_emp_length_replaced.write \
.format("parquet") \
.mode("overwrite") \
.option("path","/user/itv014478/lendingclubproject/cleaned/customer_data") \
.save()